In [1]:
import lib.NotationLinear as nlin
import lib.score_comparison_lib as scl
import importlib

import music21 as m21
from pathlib import Path

music21: Certain music21 functions might need the optional package matplotlib;
                  if you run into errors, install it by following the instructions at
                  http://mit.edu/music21/doc/installing/installAdditional.html


In [2]:
score1_path = Path("test_scores/musicxml/longHaydn_a.musicxml")
score1 = m21.converter.parse(str(score1_path))
score2_path = Path("test_scores/musicxml/longHaydn_b.musicxml")
score2 = m21.converter.parse(str(score2_path))
# build ScoreTrees
score_lin1 = nlin.Score(score1)
score_lin2 = nlin.Score(score2)

In [3]:
importlib.reload(scl)
# compute the non common_subsequences for part 0
scl.non_common_subsequences_myers([[0,0],[1,1],[2,2],[3,3]],[[1,1],[1,1],[3,3]])

[{'original': [0], 'compare_to': []}, {'original': [2], 'compare_to': [1]}]

In [8]:
%%time
# compute the non common_subsequences for part 0
part = 0
length = -1
seq1 = score_lin1.part_list[part].bar_list[:length]
seq2 = score_lin2.part_list[part].bar_list[:length]
print("About to compute LCS for sequences of length", len(seq1), len(seq2))
non_common_subsequences = scl.non_common_subsequences_of_measures(
    seq1, seq2
)
print(non_common_subsequences)

About to compute LCS for sequences of length 521 521
[{'original': [[[[('G5', 'None', False)],4,0,[],[],2008959198256,[],[], [('D5', 'None', False)],4,0,[],[],2008959198400,[],[], [('R', 'None', False)],4,0,[],[],2008959198544,[],[], [('D5', 'None', False)],4,0,['start', 'start'],[],2008959198592,[],[], [('C5', 'None', False)],4,0,['continue', 'continue'],[],2008959198784,[],[], [('B4', 'None', False)],4,0,['continue', 'continue'],[],2008959198976,[],[], [('A4', 'None', False)],4,0,['stop', 'stop'],[],2008959199168,[],[]]]], 'compare_to': [[[[('G5', 'None', False)],4,0,[],[],2009020743632,[],[], [('D5', 'None', False)],4,0,[],[],2009020741616,[],[], [('R', 'None', False)],4,0,[],[],2009020740656,[],[], [('D5', 'sharp', False)],4,0,['start', 'start'],[],2009020739648,[],[], [('C5', 'None', False)],4,0,['continue', 'continue'],[],2009020742384,[],[], [('B4', 'None', False)],4,0,['continue', 'continue'],[],2009020741328,[],[], [('A4', 'None', False)],4,0,['stop', 'stop'],[],2009020739936,

In [8]:
non_common_subsequences

[{'original': [[[[('G5', 'None', False)],4,0,[],[],3126091008464,[],[], [('D5', 'None', False)],4,0,[],[],3126091008608,[],[], [('R', 'None', False)],4,0,[],[],3126091008752,[],[], [('D5', 'None', False)],4,0,['start', 'start'],[],3126091008800,[],[], [('C5', 'None', False)],4,0,['continue', 'continue'],[],3126091008992,[],[], [('B4', 'None', False)],4,0,['continue', 'continue'],[],3126091009184,[],[], [('A4', 'None', False)],4,0,['stop', 'stop'],[],3126091009376,[],[]]]],
  'compare_to': [[[[('G5', 'None', False)],4,0,[],[],3126153285584,[],[], [('D5', 'None', False)],4,0,[],[],3126153223904,[],[], [('R', 'None', False)],4,0,[],[],3126153222896,[],[], [('D5', 'sharp', False)],4,0,['start', 'start'],[],3126153222848,[],[], [('C5', 'None', False)],4,0,['continue', 'continue'],[],3126153221456,[],[], [('B4', 'None', False)],4,0,['continue', 'continue'],[],3126153222176,[],[], [('A4', 'None', False)],4,0,['stop', 'stop'],[],3126153166560,[],[]]]]}]

4496846402853166584

In [13]:
import numpy as np
np.array([[1,2],[3,4]])

array([[1, 2],
       [3, 4]])

In [17]:
a = np.array([[1,2],[3,4],[5,6]])
len(a)

3

In [20]:
from collections import namedtuple
import sys


# See frontier in myers_diff
Frontier = namedtuple('Frontier', ['x', 'history'])

def myers_diff(a_lines, b_lines):
    """
    An implementation of the Myers diff algorithm.
    See http://www.xmailserver.org/diff2.pdf
    """
    # This marks the farthest-right point along each diagonal in the edit
    # graph, along with the history that got it there
    frontier = {1: Frontier(0, [])}

    def one(idx):
        """
        The algorithm Myers presents is 1-indexed; since Python isn't, we
        need a conversion.
        """
        return idx - 1

    a_max = len(a_lines)
    b_max = len(b_lines)
    for d in range(0, a_max + b_max + 1):
        for k in range(-d, d + 1, 2):
            # This determines whether our next search point will be going down
            # in the edit graph, or to the right.
            #
            # The intuition for this is that we should go down if we're on the
            # left edge (k == -d) to make sure that the left edge is fully
            # explored.
            #
            # If we aren't on the top (k != d), then only go down if going down
            # would take us to territory that hasn't sufficiently been explored
            # yet.
            go_down = (k == -d or 
                    (k != d and frontier[k - 1].x < frontier[k + 1].x))

            # Figure out the starting point of this iteration. The diagonal
            # offsets come from the geometry of the edit grid - if you're going
            # down, your diagonal is lower, and if you're going right, your
            # diagonal is higher.
            if go_down:
                old_x, history = frontier[k + 1]
                x = old_x
            else:
                old_x, history = frontier[k - 1]
                x = old_x + 1

            # We want to avoid modifying the old history, since some other step
            # may decide to use it.
            history = history[:]
            y = x - k

            # We start at the invalid point (0, 0) - we should only start building
            # up history when we move off of it.
            if 1 <= y <= b_max and go_down:
                history.append(("Insert",b_lines[one(y)]))
            elif 1 <= x <= a_max:
                history.append(("Remove",a_lines[one(x)]))

            # Chew up as many diagonal moves as we can - these correspond to common lines,
            # and they're considered "free" by the algorithm because we want to maximize
            # the number of these in the output.
            while x < a_max and y < b_max and a_lines[one(x + 1)] == b_lines[one(y + 1)]:
                x += 1
                y += 1
                history.append(("Keep",a_lines[one(x)]))

            if x >= a_max and y >= b_max:
                # If we're here, then we've traversed through the bottom-left corner,
                # and are done.
                return history
            else:
                frontier[k] = Frontier(x, history)

    assert False, 'Could not find edit script'

In [24]:
out = myers_diff([[1,1],[2,2],[4,4],[7,7],[7,7]], [[2,2],[3,3],[4,4],[5,5]])
out[0]

('Remove', [1, 1])

In [16]:
diff([1,2,4,7,7,7,4], [2,3,4,5])

[(0, 1, 0, 0), (2, 0, 1, 1), (3, 4, 3, 1)]